In [1]:
# Split data into individual rasters
import netCDF4 as nc
import rasterio
from rasterio.transform import from_origin
import psycopg2
from datetime import datetime

conn = psycopg2.connect(database="postgres", user="postgres", password="Passwordd", host="localhost", port="5432")
cursor = conn.cursor()

# Replace 'your_netcdf_file.nc' with your NetCDF file path
nc_file = './downloaded_files/2023/ncdd-202312-grd-scaled.nc'


# Open the NetCDF file
dataset = nc.Dataset(nc_file)

# Fetch the dimensions
time = dataset['time']
lat = dataset['lat'][:]
lon = dataset['lon'][:]

# Calculate the transformation for raster
transform = from_origin(lon.min(), lat.max(), lon[1] - lon[0], lat[1] - lat[0])

# Process each type of data
for data_type in ['tmin', 'tmax', 'tavg', 'prcp']:
    variable = dataset.variables[data_type]
    for i, t in enumerate(time[:]):
        print(i)
        # Convert time number to actual datetime
        dt = nc.num2date(t, time.units)
        date = datetime(dt.year, dt.month, dt.day, dt.hour, dt.minute, dt.second)
        
        # Create a raster file for each time slice
        raster_filename = f"./downloaded_files/2023/split/{data_type}_{date.strftime('%Y%m%d%H%M%S')}.tif"
        with rasterio.open(
            raster_filename, 'w', driver='GTiff',
            height=variable.shape[1], width=variable.shape[2],
            count=1, dtype=str(variable.dtype),
            crs='+proj=latlong', transform=transform
        ) as raster:
            raster.write(variable[i, :, :], 1)

        # Load raster into PostGIS (make sure you have PostGIS extensions and raster support enabled)
        cursor.execute(
            "INSERT INTO grid_data (rast, time, datatype) VALUES (ST_FromGDALRaster(%s), %s, %s)",
            (open(raster_filename, 'rb').read(), date, data_type)
        )
        conn.commit()

# Close the dataset and database connection
dataset.close()
conn.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [73]:
# Connect to your PostgreSQL database
conn = psycopg2.connect(database="postgres", user="postgres", password="Passwordd", host="localhost", port="5432")
cursor = conn.cursor()

# Create tables in database
state = "CREATE EXTENSION IF NOT EXISTS postgis;"
cursor.execute(state)
conn.commit()
state = "CREATE EXTENSION IF NOT EXISTS postgis_raster;"
cursor.execute(state)
conn.commit()

state = """CREATE TABLE IF NOT EXISTS grid_data (
    id serial PRIMARY KEY,
    rast raster,
    time timestamp without time zone,
    datatype VARCHAR(255)
);"""
cursor.execute(state)

conn.commit()
conn.close()